In [ ]:
import pennylane as qml
import numpy as np
from scipy.linalg import expm,sinm,cosm
from numpy.linalg import solve
%run ISING_INSPIRED_H.ipynb

In [6]:
def condition_number(matrix):
    # Compute eigenvalues and eigenvectors of the matrix
    eigv, eigc = np.linalg.eig(matrix)
    
    # Calculate the absolute values of the eigenvalues
    abs_eigv = [abs(i) for i in eigv]
    
    # Compute the condition number as the ratio of the largest to the smallest eigenvalue
    cond_number = max(abs_eigv) / min(abs_eigv)
    
    # Return the condition number and the smallest eigenvalue in magnitude
    return cond_number, min(abs_eigv)

### Generate the a list of quantum krylov-subspace basis.

In [7]:
def Quantum_Krylov_subsapce(dimension,matrix,vector,time_step):
    Qkrylov_subspace=[]
    for i in range(dimension):
        Qkrylov_subspace.append(np.dot(expm(-1j*i*time_step*matrix),vector))
    return(Qkrylov_subspace)

### Construct the matrix $\mathcal{F}(\mathcal{H})$ and the vector $\mathcal{s}$ based on Eq. (7) in the text.

In [8]:
def matrix_F(QK_space,matrix):
    matrix_F=[]
    for i in range(len(QK_space)):
        for j in range(len(QK_space)):
            matrix_F.append(np.inner(QK_space[i].T.conjugate(),np.dot(matrix,QK_space[j])))
    return np.array(matrix_F).reshape(len(QK_space),len(QK_space))
def matrix_S(QK_space):
    matrix_S=[]
    for i in range(len(QK_space)):
        matrix_S.append(np.inner(QK_space[i].T.conjugate(),QK_space[0]))
    return np.array(matrix_S)

### Calculate the approximate solution

In [9]:
def QK_solution(dimension,matrix,vector,time_step):
    quantum_krylov_subspace=Quantum_Krylov_subsapce(dimension,matrix,vector,time_step)
    M_h=matrix_F(quantum_krylov_subspace,matrix)
    M_s=matrix_S(quantum_krylov_subspace)
    solver1=solve(M_h,M_s.T)
    solver2=np.zeros(len(vector))[1]
    for index,element in enumerate(quantum_krylov_subspace):
        solver2=solver2 + solver1[index]*element
    return (np.dot(matrix,solver2))

### An example from the paper

In [11]:
n=10
H=Ising_inspired_Hamiltonian(n, 0.1,1,10*(2.02487285094645)*(5.262813667752477))
U=qml.matrix(H)
b=np.ones((2**n,2**n)[0])*(1/np.sqrt(2**n))
Error=[]
for i in range(2,33):
    solution=QK_solution(i,U,b,0.001)
    Error.append(abs(1-np.inner(solution,b)))

### Further discussion on the impact of the accuracy of Eq. (7) on the final solution. This part of the numerical simulation corresponds to Fig. 4 in the paper. 
Here, we utilize the truncation values provided by NumPy to construct the matrix $ \mathcal{F}(\mathcal{H})$ and the vector $ \mathbf{s} $ respectively, thereby investigating the impact of the accuracy of Equation (7) on the final results.

In [12]:
def matrix_F_truncated(QK_space,matrix,truncated_precision):
    matrix_H=[]
    for i in range(len(QK_space)):
        for j in range(len(QK_space)):
            matrix_H.append(np.inner(QK_space[i].T.conjugate(),np.dot(matrix,QK_space[j])))
    return np.round(np.array(matrix_H).reshape(len(QK_space),len(QK_space)),truncated_precision)
def matrix_S_truncated(QK_space,truncated_precision):
    matrix_S=[]
    for i in range(len(QK_space)):
        matrix_S.append(np.inner(QK_space[i].T.conjugate(),QK_space[0]))
    return np.round(np.array(matrix_S),truncated_precision)
def QK_solution_truncated(dimension,matrix,vector,time_step,truncated_precision):
    quantum_krylov_subspace=Quantum_Krylov_subsapce(dimension,matrix,vector,time_step)
    M_h=matrix_F_truncated(quantum_krylov_subspace,matrix,truncated_precision)
    M_s=matrix_S_truncated(quantum_krylov_subspace,truncated_precision)
    solver1=solve(M_h,M_s.T)
    solver2=np.zeros(len(vector))[1]
    for index,element in enumerate(quantum_krylov_subspace):
        solver2=solver2 + solver1[index]*element
    return (np.dot(matrix,solver2))

In [13]:
Truncated_Error=[]
for i in [7,8,9,10,11,12,13,14,15]:
    precision=i
    solution=QK_solution_truncated(10,U,b,0.001,precision)
    Truncated_Error.append(abs(1-np.inner(solution,b)))